In [47]:
model = models.mobilenet_v2(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
del model

In [48]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Load pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.eval()

# Define transforms to preprocess the input image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the input image
image_path = "/content/tiger.webp"
image = Image.open(image_path)
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Move the input and model to GPU for faster computation if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# Perform prediction
with torch.no_grad():
    output = model(input_batch)

# Post-process the output (convert logits to probabilities)
probabilities = torch.nn.functional.softmax(output[0], dim=0)



In [ ]:
probabilities

In [49]:
# Get the predicted class index
predicted_class_idx = torch.argmax(probabilities).item()

print("Predicted class index:", predicted_class_idx)
print("Probability:", probabilities[predicted_class_idx].item())

Predicted class index: 292
Probability: 0.9457566738128662


In [50]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

categories[predicted_class_idx]

'tiger'

In [20]:
class_labels = model.classifier[-1].state_dict()['weight'].shape[0]

print("Number of classes:", class_labels)

Number of classes: 1280


In [25]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2024-04-17 12:34:51--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-04-17 12:34:51 (73.4 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

categories

In [41]:
if "tiger" in categories:
  print("pongal")

pongal


In [55]:
cp -r /content/drive/MyDrive/mia  /content/mia

In [65]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v2

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
train_dataset = torchvision.datasets.ImageFolder(root='/content/mia', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load pretrained MobileNetV2
model = mobilenet_v2(pretrained=True)
# Modify the model's output layer for single class classification
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 1)

# Optionally freeze the pretrained layers
# for param in model.features.parameters():
#     param.requires_grad = False

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())  # Squeeze outputs if necessary
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/50], Loss: 0.9111
Epoch [2/50], Loss: 0.5661
Epoch [3/50], Loss: 0.3245
Epoch [4/50], Loss: 0.1505
Epoch [5/50], Loss: 0.0882
Epoch [6/50], Loss: 0.0555
Epoch [7/50], Loss: 0.0299
Epoch [8/50], Loss: 0.0197
Epoch [9/50], Loss: 0.0137
Epoch [10/50], Loss: 0.0093
Epoch [11/50], Loss: 0.0069
Epoch [12/50], Loss: 0.0050
Epoch [13/50], Loss: 0.0037
Epoch [14/50], Loss: 0.0029
Epoch [15/50], Loss: 0.0024
Epoch [16/50], Loss: 0.0019
Epoch [17/50], Loss: 0.0014
Epoch [18/50], Loss: 0.0014
Epoch [19/50], Loss: 0.0012
Epoch [20/50], Loss: 0.0010
Epoch [21/50], Loss: 0.0009
Epoch [22/50], Loss: 0.0008
Epoch [23/50], Loss: 0.0007
Epoch [24/50], Loss: 0.0006
Epoch [25/50], Loss: 0.0006
Epoch [26/50], Loss: 0.0005
Epoch [27/50], Loss: 0.0005
Epoch [28/50], Loss: 0.0005
Epoch [29/50], Loss: 0.0004
Epoch [30/50], Loss: 0.0004
Epoch [31/50], Loss: 0.0004
Epoch [32/50], Loss: 0.0004
Epoch [33/50], Loss: 0.0004
Epoch [34/50], Loss: 0.0003
Epoch [35/50], Loss: 0.0003
Epoch [36/50], Loss: 0.0003
E

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v2

# Define transforms with data augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset with train transforms
train_dataset = torchvision.datasets.ImageFolder(root='/content/mia', transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Modify the model's output layer for single class classification
num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 1)

# Optionally freeze the pretrained layers
# for param in model.features.parameters():
#     param.requires_grad = False

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Define number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())  # Squeeze outputs if necessary
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        predictions = (torch.sigmoid(outputs) > 0.5).int()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")


In [61]:
import torch
from torchvision import transforms
from PIL import Image



# Define transformations for input data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input image
    transforms.ToTensor(),           # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

# Load and preprocess your input data
input_image = Image.open('/content/448464640.webp')
input_tensor = transform(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Put model in evaluation mode
model.eval()

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Interpret the output
probabilities = torch.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

if predicted_class == 0:
  print("mia")
else:
  print("not mia")

mia


In [66]:
import torch
import torchvision.transforms as transforms
from PIL import Image

model.eval()  # Set the model to evaluation mode

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess your input image
image_path = '/content/mia-khalifa-grinning-zakg54fkgw8hatlm.webp'
image = Image.open(image_path)
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Forward pass
with torch.no_grad():
    outputs = model(input_tensor)


not mia


In [70]:
prediction

0.00020826698164455593

In [67]:
# Post-processing
probabilities = torch.sigmoid(outputs)  # Assuming your model output is logits, and you're using BCEWithLogitsLoss
prediction = probabilities.item()  # Assuming you're interested in a single class

# Decision
threshold = 0.5  # Adjust this threshold based on your model's behavior and your requirements
if prediction >= threshold:
    print("mia")
else:
    print("not mia")

not mia


In [77]:
import torch
from PIL import Image
import torchvision.transforms as transforms

# Load and preprocess the input image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

model.eval()

# Perform inference
input_image = preprocess_image('/content/mia-khalifa-grinning-zakg54fkgw8hatlm.webp')
with torch.no_grad():
    output = model(input_image)

# Convert logits to class predictions
prediction = (output > 0).int().item()  # Convert logits to binary prediction
if prediction == 1:
    print("The image belongs to your class.")
else:
    print("The image does not belong to your class.")


The image does not belong to your class.
